<a href="https://colab.research.google.com/github/axel-sirota/practical-nlp/blob/main/3-text-generation/Practical_NLP_10_5_Generate_with_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install textblob 'gensim==4.2.0' 'keras_nlp' 'transformers'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 24.7 MB/s 
     |████████████████████████████████| 5.9 MB 40.8 MB/s 
     |████████████████████████████████| 578.0 MB 15 kB/s 
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     |████████████████████████████████| 4.6 MB 47.8 MB/s 


In [10]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
import smart_open
import pandas as pd
import gensim
from gensim.models.callbacks import CallbackAny2Vec
from gensim.test.utils import get_tmpfile
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
from transformers import set_seed, pipeline
import keras_nlp
import os
import time
import sys
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from collections import defaultdict


TRACE = False
embedding_dim = 100
rnn_units = 128
epochs=25
buffer_size = 64
corpus_size=25000
test_corpus_size=5000
# Batch size
batch_size = 64
min_count_words = 5
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  set_seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  K.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
generator = pipeline('text-generation', model='gpt2')
query = 'Midtown Sunny 2-Bedroom'
generated_descriptions = generator(query, max_length=200, num_return_sequences=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [12]:
%%writefile get_data.sh
if [ ! -f train_corpus_descriptions_airbnb.csv ]; then
  wget -O train_corpus_descriptions_airbnb.csv https://www.dropbox.com/s/5rp7ibop99qyafo/train_corpus_descriptions_airbnb.csv?dl=0
fi


Overwriting get_data.sh


In [13]:
!bash get_data.sh

In [14]:
def preprocess_text(text, should_join=True):
    text = str(text)
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [15]:
def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = preprocess_text(line, should_join=False)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [16]:
train_corpus = list(read_corpus('./train_corpus_descriptions_airbnb.csv'))[:corpus_size]

In [18]:
doc2vec_model = gensim.models.doc2vec.Doc2Vec(vector_size=embedding_dim, min_count=2, epochs=epochs, workers=5)
doc2vec_model.build_vocab(train_corpus)
doc2vec_model.train(train_corpus, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [19]:
descriptions = []
for description in generated_descriptions:
    tokenized = list(gensim.utils.simple_preprocess(description["generated_text"]))
    similarity = doc2vec_model.similarity_unseen_docs(doc_words1=tokenized, doc_words2= list(gensim.utils.simple_preprocess(query)))
    descriptions.append({"description": description["generated_text"], "similarity": similarity})

df_desc = pd.DataFrame(descriptions).sort_values(by="similarity", ascending=False, inplace=False)
df_desc.head()

,description,similarity
7,Midtown Sunny 2-Bedroom Room 1 - 1 Bathroom 1 ...,0.376321
0,"Midtown Sunny 2-Bedroom House, 835 SE St. W., ...",0.256287
9,Midtown Sunny 2-Bedroom-1-Residence Home - 3 B...,0.234255
4,"Midtown Sunny 2-Bedroom Apartment\n\n3,600 squ...",0.233562
2,Midtown Sunny 2-Bedroom\n\n1. The Original Hou...,0.219019


In [20]:
df_desc.iloc[0].description

'Midtown Sunny 2-Bedroom Room 1 - 1 Bathroom 1 - 1\n\nBristol Village - Breslutes 1-Bedroom - 1 - Bathroom - 1\n\nBristol Village - Downtown Sunny 1-Bedroom - 1 - Bathroom - 1\n\nBristol Village - Downtown Sunny 2-Bedroom - 1 - Bathroom - 1\n\nBristol Village - Downtown Sunny 2-Bedroom - 1 - Bathroom - 2\n\nBristol Village - Downtown Sunny 2-Bedroom - 1 - Bathroom - 2\n\nBristol Village - Downtown Sunny 2-Bedroom - 2 - Bathroom - 2\n\nBristol Village - Downtown Sunny 2-Bedroom - 2 - Bathroom - 3\n\nBristol Village - Downtown Sunny 2-Bedroom - 3 - Bathroom\n\nBristol Village - Downtown Sunny 2-Bedroom - 3 - Bathroom - 3\n\nBrist'

In [21]:
df_desc.iloc[1].description

'Midtown Sunny 2-Bedroom House, 835 SE St. W., New York 12201. Hours: 9:00 a.m.-10:00 p.m., Monday to Friday. $60 plus taxes.\n\n4. Sushi Café, 1212 S. New York Ave.; 212-647-8477. Hours: 2:00 p.m.-3:00 pm. Location: Sushi Café, 1310 SE Fifth Avenue. Closed Sunday through Sunday.\n\n5. Navell House, 2511 N. Market St.; 212-349-2895. Hours: 6:00 a.m.-6:00 p.m., Monday to Friday.\n\n6. Restaurant of Eastern New York, 2750 E. Elm St.; 212-847-3500. Hours: 6:00 a.m.-6:00 p.m.\n\n7. Café San Jacinto, 2108 West Sixth Ave.; 212'